In [47]:
import os
from verification_system import VerificationSystem
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

# Eksperyment bez fine-tuningu

In [39]:
verification_system = VerificationSystem(
    database_path=os.path.join("data", "database"), model_name="ArcFace"
)

authorized_users
24-04-08 14:25:19 - Searching data\database\incoming_users\authorized_users\25\010802.jpg in 1826 length datastore
24-04-08 14:25:19 - find function duration 0.3077411651611328 seconds


In [40]:
exp = os.listdir("data/database/incoming_users")
exp_lum = list(filter(lambda x: "lum" in x, exp))
exp_psnr = list(filter(lambda x: "psnr" in x, exp))

In [41]:
lumsplit = lambda x: x.split("_")[0] + " " + x.split("_")[-2] + ": " + x.split("_")[-1]
psnrsplit = lambda x: x.split("_")[0] + ": " + x.split("_")[-1]

In [42]:
res_lum = {}
res_psnr = {}

In [45]:
for in_dir in exp_lum:
    expkey = lumsplit(in_dir)
    print("Experimenting with: ", expkey)

Experimenting with:  lum quadratic: None
Experimenting with:  lum linear: 0.5
Experimenting with:  lum linear: 0.6
Experimenting with:  lum linear: 0.75
Experimenting with:  lum linear: 1.33
Experimenting with:  lum linear: 1.5
Experimenting with:  lum constant: -100
Experimenting with:  lum constant: -20
Experimenting with:  lum constant: -10
Experimenting with:  lum constant: 30
Experimenting with:  lum quadratic: None
Experimenting with:  lum linear: 0.5
Experimenting with:  lum linear: 0.6
Experimenting with:  lum linear: 0.75
Experimenting with:  lum linear: 1.33
Experimenting with:  lum linear: 1.5
Experimenting with:  lum constant: -100
Experimenting with:  lum constant: -20
Experimenting with:  lum constant: -10
Experimenting with:  lum constant: 30


In [ ]:
for index in tqdm(iterable=range(int(len(exp_lum) / 2)), desc="Processing Luminance"):
    authorized_users_path = os.path.join(
        "data", "database", "incoming_users", exp_lum[index]
    )
    unauthorized_users_path = os.path.join(
        "data", "database", "incoming_users", exp_lum[index + int(len(exp_lum) / 2)]
    )
    expkey = lumsplit(exp_lum[index])

    try:
        df_authorized = verification_system.verify_multiple_users(authorized_users_path)
        df_unauthorized = verification_system.verify_multiple_users(
            unauthorized_users_path
        )

        false_acceptance_rate, false_rejection_rate = (
            verification_system.calculate_far_frr(df_authorized, df_unauthorized)
        )
        res_lum[expkey] = {"FAR": false_acceptance_rate, "FRR": false_rejection_rate}
    except Exception as e:
        print(f"Error: {e}")
        res_lum[expkey] = {"FAR": -1, "FRR": -1}
        continue

Processing Luminance:   0%|          | 0/10 [00:00<?, ?it/s]

In [ ]:
try:
    plt.bar(res_lum.keys(), res_lum.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different luminance levels")
    plt.xlabel("Luminance level")
    plt.ylabel("Access granted rate")
    plt.show()
except Exception as e:
    print(f"Error: {e}")
finally:
    res_lum = {k: v for k, v in res_lum.items() if v is not None}
    plt.bar(res_lum.keys(), res_lum.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different luminance levels")
    plt.xlabel("Luminance level")
    plt.ylabel("Access granted rate")
    plt.show()

In [ ]:
for in_dir in exp_psnr:
    path = os.path.join("data/database/incoming_users", in_dir)
    expkey = psnrsplit(in_dir)
    print("Experimenting with: ", expkey)
    try:
        df_multi = verification_system.verify_multiple_users(path)
        res_psnr[expkey] = verification_system.calculate_access_granted_rate(
            df_users=df_multi
        )
    except Exception as e:
        print(f"Error: {e}")
        res_psnr[expkey] = -1
        continue

In [ ]:
try:
    plt.bar(res_psnr.keys(), res_psnr.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different PSNR levels")
    plt.xlabel("PSNR level")
    plt.ylabel("Access granted rate")
    plt.show()
except Exception as e:
    print(f"Error: {e}")
finally:
    res_psnr = {k: v for k, v in res_psnr.items() if v is not None}
    plt.bar(res_psnr.keys(), res_psnr.values())
    plt.xticks(rotation=90)
    plt.title("Access granted rate for different PSNR levels")
    plt.xlabel("PSNR level")
    plt.ylabel("Access granted rate")
    plt.show()

# Eksperyment z fine-tuningu

In [ ]:
# najpierw w utils/exp_database stworzyć foldery na bazy danych z takimi parametrami jakie chcemy fintunować